In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))

    for c in ['MA_TP', 'BB_UPPER', 'BB_LOWER']:
        fig.add_trace(go.Scatter( 
            x=df_plot.time,
            y=df_plot[c],
            line=dict(width=2,color="#4290f5"),
            line_shape='spline',
            name=c,
            mode='lines'
            ))        

    df_markers = df_plot[df_plot.BB_SIGNAL!=0]
    fig.add_trace(go.Candlestick(
        x=df_markers.time, open=df_markers.mid_o, high=df_markers.mid_h, low=df_markers.mid_l, close=df_markers.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#3480eb',
        decreasing_fillcolor="#3480eb",
        increasing_line_color='#3480eb',  
        decreasing_line_color='#3480eb'
    ))

    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [3]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']

In [5]:
def Get_BB_Signal(row):
    if row.mid_c > row.BB_UPPER and row.mid_c > row.mid_o:
        return -1
    if row.mid_c < row.BB_LOWER and row.mid_c < row.mid_o:
        return 1
    return 0

In [6]:
df = df_raw[cols].copy()
df['TP'] = (df.mid_h + df.mid_l + df.mid_c) / 3
df['MA_TP'] = df.TP.rolling(window=20).mean()
df['STD_TP'] = df.TP.rolling(window=20).std()
df['BB_UPPER'] = df.MA_TP + 2 * df.STD_TP
df['BB_LOWER'] = df.MA_TP - 2 * df.STD_TP
df['BB_SIGNAL'] = df.apply(Get_BB_Signal, axis=1)

In [12]:
df_plot = df.iloc[-100:]
plot_candles(df_plot)

In [13]:
df['EMA_LONG'] = df.mid_c.ewm(span=26, min_periods=26).mean()
df['EMA_SHORT'] = df.mid_c.ewm(span=12, min_periods=12).mean()
df['MACD'] = df.EMA_SHORT - df.EMA_LONG
df['SIGNAL'] = df.MACD.ewm(span=9).mean()
df['HIST'] = df.MACD - df.SIGNAL
df.dropna(inplace=True)

In [14]:
df.head()

,time,mid_o,mid_h,mid_l,mid_c,TP,MA_TP,STD_TP,BB_UPPER,BB_LOWER,BB_SIGNAL,EMA_LONG,EMA_SHORT,MACD,SIGNAL,HIST
50,2023-01-12 06:00:00+00:00,131.637,131.902,130.854,130.959,131.238333,132.050133,0.365741,132.781615,131.318652,1,132.093736,131.962559,-0.131177,-0.131177,0.000000
51,2023-01-12 10:00:00+00:00,130.960,131.241,129.501,129.652,130.131333,131.972133,0.560683,133.093499,130.850768,1,131.886984,131.603137,-0.283847,-0.215993,-0.067853
52,2023-01-12 14:00:00+00:00,129.651,130.786,129.176,129.196,129.719333,131.847483,0.749723,133.346928,130.348038,1,131.661517,131.229331,-0.432186,-0.304597,-0.127589
53,2023-01-12 18:00:00+00:00,129.196,129.447,128.864,129.302,129.204333,131.704833,0.951863,133.608559,129.801107,0,131.465724,130.930467,-0.535258,-0.382734,-0.152524
54,2023-01-12 22:00:00+00:00,129.246,129.436,128.641,128.936,129.004333,131.565250,1.126465,133.818180,129.312320,1,131.257661,130.621568,-0.636092,-0.458102,-0.177990


In [15]:
df_plot = df.iloc[-100:]
fig = go.Figure()

fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.SIGNAL,
    line=dict(width=2,color="#03adfc"),
    line_shape='spline',
    name='SIGNAL',
    mode='lines'
    ))        
fig.add_trace(go.Scatter( 
    x=df_plot.time,
    y=df_plot.MACD,
    line=dict(width=2,color="#fc4a03"),
    line_shape='spline',
    name='MACD',
    mode='lines'
    ))         
fig.add_trace(go.Bar( 
    x=df_plot.time,
    y=df_plot.HIST,
    name='HIST'
    ))         

fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()